In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\project\\Customer Purchase Prediction'

In [4]:
from src.utils import read_yaml, create_directories
import gdown

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(Path("config.yml"))
        self.schema = read_yaml(Path("schema.yml"))

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [7]:
import os
import sys
import urllib.request as request
import zipfile
from src.logger import logging
from src.exception import CustomException
from src.utils import get_size

In [15]:
class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        file_id = self.config.source_URL.split("/")[-2]
        gdrive_url = f"https://drive.google.com/uc?id={file_id}"

        # Tạo thư mục nếu chưa tồn tại
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)

        if os.path.exists(self.config.local_data_file):
            logging.info(f"File downloaded successfully: {self.config.local_data_file}")
        else:
            logging.info(f"Downloading file from {gdrive_url} to {self.config.local_data_file}")
            gdown.download(gdrive_url, self.config.local_data_file, quiet=False)
            logging.info(f"File downloaded successfully: {self.config.local_data_file}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        # Giải nén file ZIP
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logging.info(f"Extracted all files to {unzip_path}")

In [17]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e, sys)